In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import pandas as pd
import os
import librosa

In [2]:
# Windowing: 400ms windows, 50% overlap
def create_windows(data, sample_rate):
    window_size = max(1, int(0.4 * sample_rate))  # samples per 400 ms
    hop_size = max(1, window_size // 2)

    # Make overlapping windows efficiently
    from numpy.lib.stride_tricks import sliding_window_view
    if len(data) < window_size:
        # Pad with zeros to create at least one window
        pad_width = window_size - len(data)
        data = np.pad(data, (0, pad_width), mode='constant')
    windows = sliding_window_view(data, window_shape=window_size)[::hop_size]  # shape: (num_windows, window_size)
    # Apply Hann window to reduce spectral leakage
    hann = np.hanning(window_size)
    win = windows * hann
    return win

# Feature extraction
def extract_features(signal, sample_rate):
    # --- safety & dtype ---
    x = np.asarray(signal, dtype=np.float32)
    if x.ndim == 2:
        x = x.mean(axis=1)

    # --- time-domain stats ---
    rms = np.sqrt(np.mean(x**2) + 1e-12)
    zcr = (np.sign(x[:-1]) != np.sign(x[1:])).mean() if x.size > 1 else 0.0
    mean = float(np.mean(x))
    std  = float(np.std(x))
    skew = float(np.mean((x - mean)**3) / (std**3 + 1e-12))
    kurt = float(np.mean((x - mean)**4) / (std**4 + 1e-12))
    energy = float(np.sum(x**2))

    # --- FFT / spectral stats ---
    fft = np.fft.rfft(x)
    mag = np.abs(fft) + 1e-12
    fft_mean = float(np.mean(mag))
    fft_energy = float(np.sum(mag**2))
    fft_min = float(np.min(mag))
    fft_max = float(np.max(mag))
    fft_diff = float(fft_max - fft_min)
    fft_median = float(np.median(mag))
    fft_mad = float(np.mean(np.abs(mag - fft_median)))

    freqs = np.fft.rfftfreq(len(x), d=1.0 / sample_rate)
    ps = mag**2
    ps_sum = float(ps.sum() + 1e-12)
    centroid = float((freqs * ps).sum() / ps_sum)
    spectral_rolloff = float(freqs[np.searchsorted(np.cumsum(ps), 0.85 * ps_sum, side="left")])
    spectral_flux = float(np.sum(np.diff(mag)**2)) if mag.size > 1 else 0.0
    spectral_spread = float(np.sqrt(np.sum(((freqs - centroid) ** 2) * ps) / ps_sum))
    ps_norm = ps / ps_sum
    entropy = float(-np.sum(ps_norm * np.log2(ps_norm + 1e-10)))
    band_energy = float(np.sum(ps[(freqs >= 300) & (freqs <= 3400)]) / ps_sum)

    base_feats = np.array([
        rms, zcr, mean, std, skew, kurt, energy,
        fft_mean, fft_energy, fft_min, fft_max, fft_diff,
        fft_median, fft_mad, centroid, entropy,
        spectral_rolloff, spectral_flux, band_energy, spectral_spread
    ], dtype=np.float32)
    
    # --- MFCC block (pooled) ---
    S = librosa.feature.melspectrogram(
        y=x, sr=sample_rate, n_fft=256, hop_length=128,
        n_mels=64, fmin=20, fmax=None
    )
    S_db = librosa.power_to_db(S, ref=np.max)
    mfcc = librosa.feature.mfcc(S=S_db, sr=sample_rate, n_mfcc=20) # 15 MFCCs

    # Compute robust deltas for very short sequences
    T = mfcc.shape[1]
    if T >= 3:
        # largest odd width <= T
        width = 2 * (T // 2) + 1
        d1 = librosa.feature.delta(mfcc, order=1, width=min(9, width), mode="nearest")
        d2 = librosa.feature.delta(mfcc, order=2, width=min(9, width), mode="nearest")
    else:
        # Not enough frames for a finite-difference estimate
        d1 = np.zeros_like(mfcc)
        d2 = np.zeros_like(mfcc)

    mfcc_mean = mfcc.mean(axis=1)
    mfcc_std  = mfcc.std(axis=1)
    d1_mean, d1_std = d1.mean(axis=1), d1.std(axis=1)
    d2_mean, d2_std = d2.mean(axis=1), d2.std(axis=1)

    mfcc_feats = np.concatenate(
        [mfcc_mean, mfcc_std, d1_mean, d1_std, d2_mean, d2_std],
        axis=0
    ).astype(np.float32)

    # --- return ONE flat vector ---
    return np.concatenate([base_feats, mfcc_feats], axis=0)

# MFCC column names
def mfcc_cols(n_mfcc=20):
    names = []
    names += [f"mfcc{i+1}_mean" for i in range(n_mfcc)]
    names += [f"mfcc{i+1}_std"  for i in range(n_mfcc)]
    names += [f"mfcc_delta{i+1}_mean" for i in range(n_mfcc)]
    names += [f"mfcc_delta{i+1}_std"  for i in range(n_mfcc)]
    names += [f"mfcc_delta2{i+1}_mean" for i in range(n_mfcc)]
    names += [f"mfcc_delta2{i+1}_std"  for i in range(n_mfcc)]
    return names

In [3]:
def process_wav(file_path, label, car_id):
    """
    Load a .wav file, extract features for each window,
    and return a labeled DataFrame.
    """
    # --- Load WAV ---
    sample_rate, data = wavfile.read(file_path)
    data = np.asarray(data)

    # --- Convert to float32 and scale to [-1, 1] ---
    data = data.astype(np.float32)
    max_val = np.max(np.abs(data))
    if max_val > 0:
        data /= max_val

    # --- Create windows ---
    win = create_windows(data, sample_rate)

    # --- Extract features per window ---
    features = np.array([extract_features(w, sample_rate) for w in win])

    # --- Make column names ---
    cols_base = ["RMS Energy", "Zero Crossing Rate", "Mean", "Std Dev", "Skewness", "Kurtosis", "Energy",
                                     "FFT Mean", "FFT Energy", "FFT Min", "FFT Max", "FFT Min-Max Diff", "FFT Median", "FFT MAD", "Centroid",
                                     "Entropy", "Spectral Rolloff", "Spectral Flux", "Band Energy", "Spectral Spread"]
    cols = cols_base + mfcc_cols(20)
    
    # --- Make DataFrame ---
    df = pd.DataFrame(features, columns=cols)
    df["label"] = label
    df["car_id"] = car_id
    df["file"] = os.path.basename(file_path)

    return df

In [4]:
import glob
import os

# Find all WAVs for car01 that contain "normal" in their names
normal_files = glob.glob(r"C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\\*car01*normal*.wav")

dfs = []
for f in normal_files:
    print(f"Processing {f}...")
    df_temp = process_wav(f, label="NORMAL", car_id="CAR01")
    dfs.append(df_temp)

# Combine all NORMAL recordings for car01
df_car01_normal = pd.concat(dfs, ignore_index=True)
df_car01_normal.head()

Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR01_NORMAL.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR01_NORMAL2.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR01_NORMAL3.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR01_NORMAL4.wav...


,RMS Energy,Zero Crossing Rate,Mean,Std Dev,Skewness,Kurtosis,Energy,FFT Mean,FFT Energy,FFT Min,...,mfcc_delta214_std,mfcc_delta215_std,mfcc_delta216_std,mfcc_delta217_std,mfcc_delta218_std,mfcc_delta219_std,mfcc_delta220_std,label,car_id,file
0,0.044055,0.330694,-0.001505,0.044030,-0.346043,12.073394,9.316158,2.392383,22385.056641,0.025002,...,0.646548,0.701208,0.659792,0.687388,0.644810,0.494265,0.675917,NORMAL,CAR01,CAR01_NORMAL.wav
1,0.041906,0.389039,-0.000132,0.041905,-0.106844,15.193405,8.429189,2.267004,20232.429688,0.072852,...,0.703674,0.741164,0.518084,0.850053,0.551509,0.598313,0.840880,NORMAL,CAR01,CAR01_NORMAL.wav
2,0.047874,0.423005,-0.000120,0.047874,-0.400630,12.855452,11.001245,2.531100,26516.429688,0.018464,...,0.933589,0.784207,0.432980,0.533195,0.547249,0.608863,0.719886,NORMAL,CAR01,CAR01_NORMAL.wav
3,0.042455,0.425922,-0.000122,0.042455,-0.320290,14.056445,8.651852,2.306833,20768.625000,0.047863,...,0.813993,0.805050,0.564478,0.783951,0.672937,0.633764,0.503617,NORMAL,CAR01,CAR01_NORMAL.wav
4,0.042594,0.406751,-0.000110,0.042594,-0.327752,9.033519,8.708310,2.381709,20910.302734,0.021633,...,0.699845,0.875866,0.455212,0.736160,0.457458,0.426331,0.679284,NORMAL,CAR01,CAR01_NORMAL.wav


In [5]:
# Find all WAVs for car02 that contain "normal" in their names
normal_files = glob.glob(r"C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\\*car02*normal*.wav")

dfs = []
for f in normal_files:
    print(f"Processing {f}...")
    df_temp = process_wav(f, label="NORMAL", car_id="CAR02")
    dfs.append(df_temp)

# Combine all NORMAL recordings for car02
df_car02_normal = pd.concat(dfs, ignore_index=True)
df_car02_normal.head()

Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR02_NORMAL.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR02_NORMAL2.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR02_NORMAL3.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR02_NORMAL4.wav...


,RMS Energy,Zero Crossing Rate,Mean,Std Dev,Skewness,Kurtosis,Energy,FFT Mean,FFT Energy,FFT Min,...,mfcc_delta214_std,mfcc_delta215_std,mfcc_delta216_std,mfcc_delta217_std,mfcc_delta218_std,mfcc_delta219_std,mfcc_delta220_std,label,car_id,file
0,0.077553,0.452803,-0.000421,0.077552,-0.390596,7.907289,28.869352,4.236690,69288.515625,0.041983,...,0.770023,0.570165,0.753398,0.522936,0.510967,0.578869,0.456869,NORMAL,CAR02,CAR02_NORMAL.wav
1,0.074096,0.423005,-0.000142,0.074096,-0.126762,6.543962,26.353054,3.881993,63248.628906,0.050359,...,0.412347,0.637729,0.484689,0.493167,0.439352,0.492966,0.389006,NORMAL,CAR02,CAR02_NORMAL.wav
2,0.074635,0.431340,-0.000092,0.074634,-0.226921,6.292687,26.737507,3.981838,64198.589844,0.052175,...,0.517832,0.658949,0.554710,0.523206,0.619357,0.525647,0.426106,NORMAL,CAR02,CAR02_NORMAL.wav
3,0.080994,0.440300,-0.000124,0.080994,-0.220870,6.838908,31.488007,4.305530,75601.062500,0.069555,...,0.511113,0.590884,0.602030,0.464501,0.588672,0.573458,0.499283,NORMAL,CAR02,CAR02_NORMAL.wav
4,0.076311,0.434257,-0.000149,0.076311,-0.143843,6.464432,27.952406,3.954669,67128.742188,0.051367,...,0.633694,0.759840,0.363845,0.581252,0.592555,0.591421,0.475594,NORMAL,CAR02,CAR02_NORMAL.wav


In [6]:
# Find all WAVs for car03 that contain "normal" in their names
normal_files = glob.glob(r"C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\\*car03*normal*.wav")

dfs = []
for f in normal_files:
    print(f"Processing {f}...")
    df_temp = process_wav(f, label="NORMAL", car_id="CAR03")
    dfs.append(df_temp)

# Combine all NORMAL recordings for car03
df_car03_normal = pd.concat(dfs, ignore_index=True)
df_car03_normal.head()

Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR03_NORMAL.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR03_NORMAL2.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR03_NORMAL3.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR03_NORMAL4.wav...


,RMS Energy,Zero Crossing Rate,Mean,Std Dev,Skewness,Kurtosis,Energy,FFT Mean,FFT Energy,FFT Min,...,mfcc_delta214_std,mfcc_delta215_std,mfcc_delta216_std,mfcc_delta217_std,mfcc_delta218_std,mfcc_delta219_std,mfcc_delta220_std,label,car_id,file
0,0.130433,0.401334,-0.025561,0.127904,-0.354138,8.751835,81.660866,5.951727,203517.250000,0.079403,...,0.431108,0.711075,0.860310,0.625435,0.465759,0.387167,0.463462,NORMAL,CAR03,CAR03_NORMAL.wav
1,0.102766,0.495728,-0.001309,0.102758,0.026729,17.281229,50.692551,5.217781,121736.656250,0.037516,...,0.521169,0.593211,1.045349,0.635058,0.505886,0.514963,0.592620,NORMAL,CAR03,CAR03_NORMAL.wav
2,0.119132,0.519066,-0.002085,0.119113,-0.713949,24.415869,68.123207,6.560986,164063.921875,0.137799,...,0.724659,0.672094,0.910762,0.728590,0.469342,0.528612,0.519109,NORMAL,CAR03,CAR03_NORMAL.wav
3,0.103753,0.536779,-0.001102,0.103747,-0.067837,14.229130,51.670586,5.323012,124122.835938,0.079128,...,0.781643,0.791808,1.175548,0.483198,0.434575,0.466394,0.670831,NORMAL,CAR03,CAR03_NORMAL.wav
4,0.113878,0.526151,-0.001677,0.113865,-0.323880,17.320175,62.247162,5.954649,149450.875000,0.060981,...,0.681773,0.586000,1.218332,0.606632,0.522615,0.370259,0.559453,NORMAL,CAR03,CAR03_NORMAL.wav


In [7]:
# Find all WAVs for car04 that contain "normal" in their names
normal_files = glob.glob(r"C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\\*car04*normal*.wav")

dfs = []
for f in normal_files:
    print(f"Processing {f}...")
    df_temp = process_wav(f, label="NORMAL", car_id="CAR04")
    dfs.append(df_temp)

# Combine all NORMAL recordings for car04
df_car04_normal = pd.concat(dfs, ignore_index=True)
df_car04_normal.head()

Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR04_NORMAL.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR04_NORMAL2.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR04_NORMAL3.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\NORMAL\CAR04_NORMAL4.wav...


,RMS Energy,Zero Crossing Rate,Mean,Std Dev,Skewness,Kurtosis,Energy,FFT Mean,FFT Energy,FFT Min,...,mfcc_delta214_std,mfcc_delta215_std,mfcc_delta216_std,mfcc_delta217_std,mfcc_delta218_std,mfcc_delta219_std,mfcc_delta220_std,label,car_id,file
0,0.060305,0.340071,-0.000356,0.060304,-0.038859,6.067549,17.455969,2.959724,41901.488281,0.033690,...,0.721174,0.572416,0.675128,0.470463,0.507745,0.479917,0.554294,NORMAL,CAR04,CAR04_NORMAL.wav
1,0.064353,0.284018,-0.000117,0.064353,-0.215408,6.351700,19.878105,3.069164,47707.925781,0.012095,...,0.413804,0.481989,0.695150,0.560961,0.537521,0.497812,0.588143,NORMAL,CAR04,CAR04_NORMAL.wav
2,0.057684,0.290477,-0.000138,0.057684,-0.300179,5.577962,15.971780,2.885366,38358.382812,0.062807,...,0.423095,0.497953,0.529288,0.559556,0.733992,0.454843,0.673070,NORMAL,CAR04,CAR04_NORMAL.wav
3,0.066017,0.298395,-0.000135,0.066017,-0.126017,6.682303,20.919415,3.100903,50208.054688,0.018497,...,0.574132,0.611651,0.527912,0.491386,0.457459,0.511244,0.643106,NORMAL,CAR04,CAR04_NORMAL.wav
4,0.066223,0.277349,-0.000136,0.066223,-0.128121,5.515840,21.050385,3.046556,50525.843750,0.098448,...,0.513761,0.494952,0.545272,0.531492,0.471367,0.603897,0.607686,NORMAL,CAR04,CAR04_NORMAL.wav


In [8]:
df_normal = pd.concat([df_car01_normal, df_car02_normal, df_car03_normal, df_car04_normal], ignore_index=True)
print(df_normal.head())
print(df_normal["label"].value_counts())
print(df_normal["car_id"].value_counts())

   RMS Energy  Zero Crossing Rate      Mean   Std Dev  Skewness   Kurtosis  \
0    0.044055            0.330694 -0.001505  0.044030 -0.346043  12.073394   
1    0.041906            0.389039 -0.000132  0.041905 -0.106844  15.193405   
2    0.047874            0.423005 -0.000120  0.047874 -0.400630  12.855452   
3    0.042455            0.425922 -0.000122  0.042455 -0.320290  14.056445   
4    0.042594            0.406751 -0.000110  0.042594 -0.327752   9.033519   

      Energy  FFT Mean    FFT Energy   FFT Min  ...  mfcc_delta214_std  \
0   9.316158  2.392383  22385.056641  0.025002  ...           0.646548   
1   8.429189  2.267004  20232.429688  0.072852  ...           0.703674   
2  11.001245  2.531100  26516.429688  0.018464  ...           0.933589   
3   8.651852  2.306833  20768.625000  0.047863  ...           0.813993   
4   8.708310  2.381709  20910.302734  0.021633  ...           0.699845   

   mfcc_delta215_std  mfcc_delta216_std  mfcc_delta217_std  mfcc_delta218_std  \
0    

In [9]:
# Find all WAVs for car01 that contain "faulty" in their names
faulty_files = glob.glob(r"C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\\*car01*faulty*.wav")

dfs = []
for f in faulty_files:
    print(f"Processing {f}...")
    df_temp = process_wav(f, label="FAULTY", car_id="CAR01")
    dfs.append(df_temp)

# Combine all FAULTY recordings for car01
df_car01_faulty = pd.concat(dfs, ignore_index=True)

Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR01_FAULTY.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR01_FAULTY2.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR01_FAULTY3.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR01_FAULTY4.wav...


In [10]:
# Find all WAVs for car02 that contain "faulty" in their names
faulty_files = glob.glob(r"C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\\*car02*faulty*.wav")

dfs = []
for f in faulty_files:
    print(f"Processing {f}...")
    df_temp = process_wav(f, label="FAULTY", car_id="CAR02")
    dfs.append(df_temp)

# Combine all FAULTY recordings for car02
df_car02_faulty = pd.concat(dfs, ignore_index=True)

Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR02_FAULTY.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR02_FAULTY2.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR02_FAULTY3.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR02_FAULTY4.wav...


In [11]:
# Find all WAVs for car03 that contain "faulty" in their names
faulty_files = glob.glob(r"C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\\*car03*faulty*.wav")

dfs = []
for f in faulty_files:
    print(f"Processing {f}...")
    df_temp = process_wav(f, label="FAULTY", car_id="CAR03")
    dfs.append(df_temp)

# Combine all FAULTY recordings for car03
df_car03_faulty = pd.concat(dfs, ignore_index=True)

Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR03_FAULTY.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR03_FAULTY2.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR03_FAULTY3.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR03_FAULTY4.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR03_FAULTY5.wav...


In [12]:
# Find all WAVs for car04 that contain "faulty" in their names
faulty_files = glob.glob(r"C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\\*car04*faulty*.wav")

dfs = []
for f in faulty_files:
    print(f"Processing {f}...")
    df_temp = process_wav(f, label="FAULTY", car_id="CAR04")
    dfs.append(df_temp)

# Combine all FAULTY recordings for car04
df_car04_faulty = pd.concat(dfs, ignore_index=True)

Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR04_FAULTY.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR04_FAULTY2.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR04_FAULTY3.wav...
Processing C:\\Users\\nusse\\Desktop\\EH5TinyML\\soundovertcp\\Server\\out\\FAULTY\CAR04_FAULTY4.wav...


In [13]:
df_faulty = pd.concat([df_car01_faulty, df_car02_faulty, df_car03_faulty, df_car04_faulty], ignore_index=True)
print(df_faulty.head())
print(df_faulty["label"].value_counts())
print(df_faulty["car_id"].value_counts())

   RMS Energy  Zero Crossing Rate      Mean   Std Dev  Skewness   Kurtosis  \
0    0.054623            0.401125 -0.000120  0.054623 -0.131798   7.665539   
1    0.055106            0.366118 -0.000148  0.055106  0.175470  13.363939   
2    0.048975            0.364243 -0.000164  0.048975 -0.041136   7.680035   
3    0.059789            0.395499 -0.000156  0.059788 -0.278957   9.651380   
4    0.042056            0.363826 -0.000209  0.042055 -0.090480   5.872614   

      Energy  FFT Mean    FFT Energy   FFT Min  ...  mfcc_delta214_std  \
0  14.321624  3.020677  34402.425781  0.091422  ...           0.889458   
1  14.576222  2.919835  34998.277344  0.010449  ...           1.038870   
2  11.513105  2.666736  27635.304688  0.089657  ...           0.908135   
3  17.158447  3.240396  41203.093750  0.046523  ...           0.853913   
4   8.489687  2.180815  20382.375000  0.027793  ...           0.681197   

   mfcc_delta215_std  mfcc_delta216_std  mfcc_delta217_std  mfcc_delta218_std  \
0    

In [14]:
df_all = pd.concat([df_normal, df_faulty], ignore_index=True)
df_all.to_csv('car_all_data300mswin.csv', index=False)

In [15]:
# Align columns (keep intersection of feature cols)
common = df_normal.columns.intersection(df_faulty.columns)
df_normal = df_normal[common]
df_faulty = df_faulty[common]

df_all = pd.concat([df_normal, df_faulty], ignore_index=True)
print(df_all["label"].value_counts())
print(df_all["car_id"].value_counts())

label
FAULTY    13502
NORMAL    12734
Name: count, dtype: int64
car_id
CAR03    7099
CAR02    6391
CAR01    6373
CAR04    6373
Name: count, dtype: int64


In [16]:
# Testing different window sizes
#df_all = pd.read_csv('car_all_data300mswin.csv')
df_all.head()

,RMS Energy,Zero Crossing Rate,Mean,Std Dev,Skewness,Kurtosis,Energy,FFT Mean,FFT Energy,FFT Min,...,mfcc_delta214_std,mfcc_delta215_std,mfcc_delta216_std,mfcc_delta217_std,mfcc_delta218_std,mfcc_delta219_std,mfcc_delta220_std,label,car_id,file
0,0.044055,0.330694,-0.001505,0.044030,-0.346043,12.073394,9.316158,2.392383,22385.056641,0.025002,...,0.646548,0.701208,0.659792,0.687388,0.644810,0.494265,0.675917,NORMAL,CAR01,CAR01_NORMAL.wav
1,0.041906,0.389039,-0.000132,0.041905,-0.106844,15.193405,8.429189,2.267004,20232.429688,0.072852,...,0.703674,0.741164,0.518084,0.850053,0.551509,0.598313,0.840880,NORMAL,CAR01,CAR01_NORMAL.wav
2,0.047874,0.423005,-0.000120,0.047874,-0.400630,12.855452,11.001245,2.531100,26516.429688,0.018464,...,0.933589,0.784207,0.432980,0.533195,0.547249,0.608863,0.719886,NORMAL,CAR01,CAR01_NORMAL.wav
3,0.042455,0.425922,-0.000122,0.042455,-0.320290,14.056445,8.651852,2.306833,20768.625000,0.047863,...,0.813993,0.805050,0.564478,0.783951,0.672937,0.633764,0.503617,NORMAL,CAR01,CAR01_NORMAL.wav
4,0.042594,0.406751,-0.000110,0.042594,-0.327752,9.033519,8.708310,2.381709,20910.302734,0.021633,...,0.699845,0.875866,0.455212,0.736160,0.457458,0.426331,0.679284,NORMAL,CAR01,CAR01_NORMAL.wav


In [17]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
meta_cols = [c for c in ["label","car_id","file"] if c in df_all.columns]
feature_cols = [c for c in df_all.columns if c not in meta_cols]

X = df_all[feature_cols].values
y = df_all["label"].values
groups = df_all["car_id"].values if "car_id" in df_all.columns else df_all["file"].values

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups))

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]


In [18]:
# TTrying gaussian naive bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
gnb_pred = gnb.predict(X_test)
print(classification_report(y_test, gnb_pred))

              precision    recall  f1-score   support

      FAULTY       0.99      0.19      0.32      3195
      NORMAL       0.55      1.00      0.71      3196

    accuracy                           0.59      6391
   macro avg       0.77      0.59      0.51      6391
weighted avg       0.77      0.59      0.51      6391



In [19]:
# Trying multylayer perceptron
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100,50), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)
mlp_pred = mlp.predict(X_test)
print(classification_report(y_test, mlp_pred))

              precision    recall  f1-score   support

      FAULTY       0.47      0.04      0.07      3195
      NORMAL       0.50      0.96      0.66      3196

    accuracy                           0.50      6391
   macro avg       0.48      0.50      0.36      6391
weighted avg       0.48      0.50      0.36      6391



In [20]:
# prepare data for keras
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)
# trying sequential with fully connected layers keras
import tensorflow as tf
from tensorflow import keras
# early stopping 
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train_enc, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])
# Evaluate on test set
test_loss, test_acc = model.evaluate(X_test, y_test_enc)
print(f"Test accuracy: {test_acc:.4f}")


Epoch 1/50


C:\Users\nusse\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


497/497 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6889 - loss: 53.4838 - val_accuracy: 0.6873 - val_loss: 2.8064
Epoch 2/50
497/497 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7372 - loss: 34.6399 - val_accuracy: 0.2802 - val_loss: 36.6567
Epoch 3/50
497/497 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7719 - loss: 21.0014 - val_accuracy: 0.2406 - val_loss: 50.5647
Epoch 4/50
497/497 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7923 - loss: 18.2009 - val_accuracy: 0.2061 - val_loss: 99.1209
Epoch 5/50
497/497 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7899 - loss: 19.3918 - val_accuracy: 0.1920 - val_loss: 123.8958
Epoch 6/50
497/497 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8144 - loss: 15.7640 - val_accuracy: 0.2053 - val_loss: 93.2322
Epoch 7/50
497/497 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8318 - loss: 12.3349 - val_accuracy: 0.1353 - val_loss: 180.8165
Epoch 8/50
497/497 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8512 - loss: 8.6430 - val_accuracy

In [21]:
# Trying random forest classifier with different hyperparameters
max_depths = [2, 3, 5, 12, 14, 16, 20]
n_estimators = [200, 300]

for depth in max_depths:
    for n_est in n_estimators:
        print(f"Training RandomForest with max_depth={depth}, n_estimators={n_est}...")
        rf = RandomForestClassifier(
            n_estimators=n_est, max_depth=depth,
            class_weight="balanced", random_state=42
        )
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)
        print(classification_report(y_test, y_pred))


Training RandomForest with max_depth=2, n_estimators=200...
              precision    recall  f1-score   support

      FAULTY       0.53      1.00      0.69      3195
      NORMAL       0.97      0.11      0.19      3196

    accuracy                           0.55      6391
   macro avg       0.75      0.55      0.44      6391
weighted avg       0.75      0.55      0.44      6391

Training RandomForest with max_depth=2, n_estimators=300...
              precision    recall  f1-score   support

      FAULTY       0.52      0.99      0.68      3195
      NORMAL       0.90      0.08      0.15      3196

    accuracy                           0.54      6391
   macro avg       0.71      0.54      0.41      6391
weighted avg       0.71      0.54      0.41      6391

Training RandomForest with max_depth=3, n_estimators=200...
              precision    recall  f1-score   support

      FAULTY       0.54      0.95      0.69      3195
      NORMAL       0.81      0.21      0.33      3196

  

In [22]:
# Trying an extra trees classifier with different hyperparameters
max_depths = [2, 3, 5, 12, 14, 16, 20]
n_estimators = [200, 300]
# import extra trees
from sklearn.ensemble import ExtraTreesClassifier
for depth in max_depths:
    for n_est in n_estimators:
        print(f"Training ExtraTrees with max_depth={depth}, n_estimators={n_est}...")
        et = ExtraTreesClassifier(
            n_estimators=n_est, max_depth=depth,
            class_weight="balanced", random_state=42
        )
        et.fit(X_train, y_train)
        y_pred = et.predict(X_test)
        print(classification_report(y_test, y_pred))
        # feature importance
        feature_importances = et.feature_importances_
        # print feature importance
        feat_ranking = sorted(zip(feature_cols, feature_importances), key=lambda x: x[1], reverse=True)
        for name, score in feat_ranking[:15]:
            print(f"{name:20s}  {score:.4f}")

Training ExtraTrees with max_depth=2, n_estimators=200...
              precision    recall  f1-score   support

      FAULTY       0.61      0.93      0.74      3195
      NORMAL       0.85      0.42      0.56      3196

    accuracy                           0.67      6391
   macro avg       0.73      0.67      0.65      6391
weighted avg       0.73      0.67      0.65      6391

Spectral Flux         0.0581
Zero Crossing Rate    0.0471
mfcc18_mean           0.0455
mfcc20_mean           0.0399
mfcc9_mean            0.0381
mfcc2_mean            0.0344
mfcc8_mean            0.0334
Energy                0.0307
Kurtosis              0.0286
FFT MAD               0.0281
Centroid              0.0268
mfcc14_mean           0.0264
mfcc3_mean            0.0255
Spectral Spread       0.0251
Band Energy           0.0246
Training ExtraTrees with max_depth=2, n_estimators=300...
              precision    recall  f1-score   support

      FAULTY       0.58      0.92      0.71      3195
      NORMAL 

In [23]:
# Trying a decision tree classifier with different hyperparameters
# import decision tree
from sklearn.tree import DecisionTreeClassifier
max_depths = [2, 3, 5, 10, 12, 14, 16, 20, 40]
for depth in max_depths:
    print(f"Training DecisionTree with max_depth={depth}...")
    dtc = DecisionTreeClassifier(
        max_depth=depth,
        class_weight="balanced", random_state=42
    )
    dtc.fit(X_train, y_train)
    y_pred = dtc.predict(X_test)
    print(classification_report(y_test, y_pred))
    # feature importances
    importances = dtc.feature_importances_
    feat_ranking = sorted(zip(feature_cols, importances), key=lambda x: x[1], reverse=True)
    for name, score in feat_ranking[:10]:
        print(f"{name:20s}  {score:.4f}")

Training DecisionTree with max_depth=2...
              precision    recall  f1-score   support

      FAULTY       0.02      0.01      0.01      3195
      NORMAL       0.43      0.74      0.54      3196

    accuracy                           0.37      6391
   macro avg       0.22      0.37      0.28      6391
weighted avg       0.22      0.37      0.28      6391

Zero Crossing Rate    0.5038
mfcc20_mean           0.4932
Spectral Flux         0.0030
RMS Energy            0.0000
Mean                  0.0000
Std Dev               0.0000
Skewness              0.0000
Kurtosis              0.0000
Energy                0.0000
FFT Mean              0.0000
Training DecisionTree with max_depth=3...
              precision    recall  f1-score   support

      FAULTY       0.02      0.01      0.01      3195
      NORMAL       0.43      0.74      0.54      3196

    accuracy                           0.37      6391
   macro avg       0.22      0.37      0.28      6391
weighted avg       0.22    